In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay1")

okay1


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [3]:
import time
import random

def specialModulus(a):
    ds=digitsum(2,a)
    return 2*8^ds

def p2RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        o2=order(2,a)
        purepart=2^o2
        oddpart=a/purepart
        d=sigma(oddpart,k=11)
        term=d*8^o2+sm*r      
        ans=ans+term*x^a
    return D(ans)

def recipP2(n):
    ans=L(1/p2RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p2RecipPowerConstantTerm(k):
    g=recipP2(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1010)

tests=[]
start=time.time()
for a in [1..1000]: #default precision must be > max a.
    c=p2RecipPowerConstantTerm(a)
    o2=order(2,c)
    ds=digitsum(2,a)
    finish=time.time()
    test=o2-3*ds
    tests=tests+[test]
    print((a,ds,o2,test))
print(tests)

(1, 1, 3, 0)
(2, 1, 3, 0)
(3, 2, 6, 0)
(4, 1, 3, 0)
(5, 2, 6, 0)
(6, 2, 6, 0)
(7, 3, 9, 0)
(8, 1, 3, 0)
(9, 2, 6, 0)
(10, 2, 6, 0)
(11, 3, 9, 0)
(12, 2, 6, 0)
(13, 3, 9, 0)
(14, 3, 9, 0)
(15, 4, 12, 0)
(16, 1, 3, 0)
(17, 2, 6, 0)
(18, 2, 6, 0)
(19, 3, 9, 0)
(20, 2, 6, 0)
(21, 3, 9, 0)
(22, 3, 9, 0)
(23, 4, 12, 0)
(24, 2, 6, 0)
(25, 3, 9, 0)
(26, 3, 9, 0)
(27, 4, 12, 0)
(28, 3, 9, 0)
(29, 4, 12, 0)
(30, 4, 12, 0)
(31, 5, 15, 0)
(32, 1, 3, 0)
(33, 2, 6, 0)
(34, 2, 6, 0)
(35, 3, 9, 0)
(36, 2, 6, 0)
(37, 3, 9, 0)
(38, 3, 9, 0)
(39, 4, 12, 0)
(40, 2, 6, 0)
(41, 3, 9, 0)
(42, 3, 9, 0)
(43, 4, 12, 0)
(44, 3, 9, 0)
(45, 4, 12, 0)
(46, 4, 12, 0)
(47, 5, 15, 0)
(48, 2, 6, 0)
(49, 3, 9, 0)
(50, 3, 9, 0)
(51, 4, 12, 0)
(52, 3, 9, 0)
(53, 4, 12, 0)
(54, 4, 12, 0)
(55, 5, 15, 0)
(56, 3, 9, 0)
(57, 4, 12, 0)
(58, 4, 12, 0)
(59, 5, 15, 0)
(60, 4, 12, 0)
(61, 5, 15, 0)
(62, 5, 15, 0)
(63, 6, 18, 0)
(64, 1, 3, 0)
(65, 2, 6, 0)
(66, 2, 6, 0)
(67, 3, 9, 0)
(68, 2, 6, 0)
(69, 3, 9, 0)
(70, 3, 9, 0)
(71, 4,

(529, 3, 9, 0)
(530, 3, 9, 0)
(531, 4, 12, 0)
(532, 3, 9, 0)
(533, 4, 12, 0)
(534, 4, 12, 0)
(535, 5, 15, 0)
(536, 3, 9, 0)
(537, 4, 12, 0)
(538, 4, 12, 0)
(539, 5, 15, 0)
(540, 4, 12, 0)
(541, 5, 15, 0)
(542, 5, 15, 0)
(543, 6, 18, 0)
(544, 2, 6, 0)
(545, 3, 9, 0)
(546, 3, 9, 0)
(547, 4, 12, 0)
(548, 3, 9, 0)
(549, 4, 12, 0)
(550, 4, 12, 0)
(551, 5, 15, 0)
(552, 3, 9, 0)
(553, 4, 12, 0)
(554, 4, 12, 0)
(555, 5, 15, 0)
(556, 4, 12, 0)
(557, 5, 15, 0)
(558, 5, 15, 0)
(559, 6, 18, 0)
(560, 3, 9, 0)
(561, 4, 12, 0)
(562, 4, 12, 0)
(563, 5, 15, 0)
(564, 4, 12, 0)
(565, 5, 15, 0)
(566, 5, 15, 0)
(567, 6, 18, 0)
(568, 4, 12, 0)
(569, 5, 15, 0)
(570, 5, 15, 0)
(571, 6, 18, 0)
(572, 5, 15, 0)
(573, 6, 18, 0)
(574, 6, 18, 0)
(575, 7, 21, 0)
(576, 2, 6, 0)
(577, 3, 9, 0)
(578, 3, 9, 0)
(579, 4, 12, 0)
(580, 3, 9, 0)
(581, 4, 12, 0)
(582, 4, 12, 0)
(583, 5, 15, 0)
(584, 3, 9, 0)
(585, 4, 12, 0)
(586, 4, 12, 0)
(587, 5, 15, 0)
(588, 4, 12, 0)
(589, 5, 15, 0)
(590, 5, 15, 0)
(591, 6, 18, 0)
(592, 3

In [4]:
import time
import random

def specialModulus(a):
    ds=digitsum(2,a)
    return 2*8^ds

def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        o2=order(2,a)
        purepart=2^o2
        oddpart=a/purepart
        d=sigma(oddpart,k=11)
        term=d*8^o2+r*2^8      
        ans=ans+term*x^a
    return D(ans)

def recipP2(n):
    ans=L(1/p2RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p2RecipPowerConstantTerm(k):
    g=recipP2(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1010)

tests=[]
start=time.time()
for a in [1..1000]: #default precision must be > max a.
    c=p2RecipPowerConstantTerm(a)
    o2=order(2,c)
    ds=digitsum(2,a)
    finish=time.time()
    test=o2-3*ds
    tests=tests+[test]
    print((a,specialModulus(a)/2^8,o2,ds,test))
print(tests)

(1, 1/16, 3, 1, 0)
(2, 1/16, 3, 1, 0)
(3, 1/2, 6, 2, 0)
(4, 1/16, 3, 1, 0)
(5, 1/2, 6, 2, 0)
(6, 1/2, 6, 2, 0)
(7, 4, 9, 3, 0)
(8, 1/16, 3, 1, 0)
(9, 1/2, 6, 2, 0)
(10, 1/2, 6, 2, 0)
(11, 4, 9, 3, 0)
(12, 1/2, 6, 2, 0)
(13, 4, 9, 3, 0)
(14, 4, 9, 3, 0)
(15, 32, 12, 4, 0)
(16, 1/16, 3, 1, 0)
(17, 1/2, 6, 2, 0)
(18, 1/2, 6, 2, 0)
(19, 4, 9, 3, 0)
(20, 1/2, 6, 2, 0)
(21, 4, 9, 3, 0)
(22, 4, 9, 3, 0)
(23, 32, 12, 4, 0)
(24, 1/2, 6, 2, 0)
(25, 4, 9, 3, 0)
(26, 4, 9, 3, 0)
(27, 32, 12, 4, 0)
(28, 4, 9, 3, 0)
(29, 32, 12, 4, 0)
(30, 32, 12, 4, 0)
(31, 256, 15, 5, 0)
(32, 1/16, 3, 1, 0)
(33, 1/2, 6, 2, 0)
(34, 1/2, 6, 2, 0)
(35, 4, 9, 3, 0)
(36, 1/2, 6, 2, 0)
(37, 4, 9, 3, 0)
(38, 4, 9, 3, 0)
(39, 32, 12, 4, 0)
(40, 1/2, 6, 2, 0)
(41, 4, 9, 3, 0)
(42, 4, 9, 3, 0)
(43, 32, 12, 4, 0)
(44, 4, 9, 3, 0)
(45, 32, 12, 4, 0)
(46, 32, 12, 4, 0)
(47, 256, 15, 5, 0)
(48, 1/2, 6, 2, 0)
(49, 4, 9, 3, 0)
(50, 4, 9, 3, 0)
(51, 32, 12, 4, 0)
(52, 4, 9, 3, 0)
(53, 32, 12, 4, 0)
(54, 32, 12, 4, 0)
(55, 256, 15,

(412, 256, 15, 5, 0)
(413, 2048, 18, 6, 0)
(414, 2048, 18, 6, 0)
(415, 16384, 21, 7, 0)
(416, 4, 9, 3, 0)
(417, 32, 12, 4, 0)
(418, 32, 12, 4, 0)
(419, 256, 15, 5, 0)
(420, 32, 12, 4, 0)
(421, 256, 15, 5, 0)
(422, 256, 15, 5, 0)
(423, 2048, 18, 6, 0)
(424, 32, 12, 4, 0)
(425, 256, 15, 5, 0)
(426, 256, 15, 5, 0)
(427, 2048, 18, 6, 0)
(428, 256, 15, 5, 0)
(429, 2048, 18, 6, 0)
(430, 2048, 18, 6, 0)
(431, 16384, 21, 7, 0)
(432, 32, 12, 4, 0)
(433, 256, 15, 5, 0)
(434, 256, 15, 5, 0)
(435, 2048, 18, 6, 0)
(436, 256, 15, 5, 0)
(437, 2048, 18, 6, 0)
(438, 2048, 18, 6, 0)
(439, 16384, 21, 7, 0)
(440, 256, 15, 5, 0)
(441, 2048, 18, 6, 0)
(442, 2048, 18, 6, 0)
(443, 16384, 21, 7, 0)
(444, 2048, 18, 6, 0)
(445, 16384, 21, 7, 0)
(446, 16384, 21, 7, 0)
(447, 131072, 24, 8, 0)
(448, 4, 9, 3, 0)
(449, 32, 12, 4, 0)
(450, 32, 12, 4, 0)
(451, 256, 15, 5, 0)
(452, 32, 12, 4, 0)
(453, 256, 15, 5, 0)
(454, 256, 15, 5, 0)
(455, 2048, 18, 6, 0)
(456, 32, 12, 4, 0)
(457, 256, 15, 5, 0)
(458, 256, 15, 5, 0)


(800, 4, 9, 3, 0)
(801, 32, 12, 4, 0)
(802, 32, 12, 4, 0)
(803, 256, 15, 5, 0)
(804, 32, 12, 4, 0)
(805, 256, 15, 5, 0)
(806, 256, 15, 5, 0)
(807, 2048, 18, 6, 0)
(808, 32, 12, 4, 0)
(809, 256, 15, 5, 0)
(810, 256, 15, 5, 0)
(811, 2048, 18, 6, 0)
(812, 256, 15, 5, 0)
(813, 2048, 18, 6, 0)
(814, 2048, 18, 6, 0)
(815, 16384, 21, 7, 0)
(816, 32, 12, 4, 0)
(817, 256, 15, 5, 0)
(818, 256, 15, 5, 0)
(819, 2048, 18, 6, 0)
(820, 256, 15, 5, 0)
(821, 2048, 18, 6, 0)
(822, 2048, 18, 6, 0)
(823, 16384, 21, 7, 0)
(824, 256, 15, 5, 0)
(825, 2048, 18, 6, 0)
(826, 2048, 18, 6, 0)
(827, 16384, 21, 7, 0)
(828, 2048, 18, 6, 0)
(829, 16384, 21, 7, 0)
(830, 16384, 21, 7, 0)
(831, 131072, 24, 8, 0)
(832, 4, 9, 3, 0)
(833, 32, 12, 4, 0)
(834, 32, 12, 4, 0)
(835, 256, 15, 5, 0)
(836, 32, 12, 4, 0)
(837, 256, 15, 5, 0)
(838, 256, 15, 5, 0)
(839, 2048, 18, 6, 0)
(840, 32, 12, 4, 0)
(841, 256, 15, 5, 0)
(842, 256, 15, 5, 0)
(843, 2048, 18, 6, 0)
(844, 256, 15, 5, 0)
(845, 2048, 18, 6, 0)
(846, 2048, 18, 6, 0)
(